In [ ]:
import cobra
import pandas as pd
import numpy as np

In [ ]:
new_model = cobra.io.read_sbml_model('Models/Recon2_Influenza_consistent.xml')

In [ ]:
core_table=pd.read_csv('/csvs/Influenza_assembled.csv', sep=';')

In [ ]:
print(len(core_table))
core_table.head()

In [ ]:
core_table = core_table.rename(columns={'Unnamed: 0': 'index'})
core_table.set_index('index', inplace=True)
core_table.head()

In [ ]:
for column in core_table.columns:
    core_table.loc['biomass_reaction'][column] = 1

In [ ]:
vbof_df = pd.DataFrame(data = [[1]* len(core_table.columns)],index = ['VBOF'],columns = core_table.columns)

In [ ]:
core_table = core_table.append([vbof_df])
print(len(core_table))

In [ ]:
#Create a dictionary that directly retreives the reaction number based on the reaction-ID
useful_dict = {}
for i in range(len(new_model.reactions)):
    useful_dict[new_model.reactions[i].id] = i+1

In [ ]:
#Counting single cells that don't have enough core reactions  
not_enough = []
solved = []

In [ ]:
#Iterate over every reaction
for col in core_table.columns:
    if core_table[col].sum() > 30:
        column = col
        core_indices = []
        for index in core_table.index:
            try:
                if core_table.loc[index][column] == 1:
                    try:
                        core_indices.append(useful_dict[index])
                    except:
                        continue
            except:
                print(column, index)
        if len(core_indices) >= 30:
            with open('CoreReactsInfluenza/'+column+'.csv', 'w', newline='') as f:
                for item in core_indices:
                    f.write(str(item)+'\n')
        else:
            not_enough.append(column)
    else:
        not_enough.append(column)
    solved.append(column)